## Importing all packages

In [27]:
import requests
from bs4 import BeautifulSoup  
import pandas as pd
from matplotlib import pyplot as plt
import openpyxl
import numpy as np

## Opioid Dispensing Rate
### - Scraping one table to get code working

In [48]:
website_url= 'https://www.cdc.gov/drugoverdose/maps/rxstate2019.html'
response = requests.get(website_url)
response.status_code

200

In [49]:
soup = BeautifulSoup(response.content, 'lxml')
print(soup.title)

<title>U.S. State Opioid Dispensing Rates, 2019  | Drug Overdose | CDC Injury Center </title>


In [50]:
tables = soup.find_all('table', attrs = {'class': 'table table-striped'})  #filters table down even more-drill 
#tables

In [51]:
type(tables)

bs4.element.ResultSet

In [52]:
len(tables)

1

In [53]:
result_list = pd.read_html(str(tables[0])) # a list of dataframes
len(result_list)

1

In [54]:
type(result_list)

list

In [55]:
world_soups = result_list[0]  # get the first df from the list
world_soups.head()

,State,Abbreviation,Opioid Dispensing Rate per 100
0,United States,US,46.7
1,Alaska,AK,39.1
2,Alabama,AL,85.8
3,Arkansas,AR,80.9
4,Arizona,AZ,44.1


In [ ]:
world_soups['Year']=2019

In [ ]:
world_soups.head()

### - Scraping all years 

In [ ]:
all_years = pd.DataFrame()
for year in range (2006,2020):
    cdc_url = f'https://www.cdc.gov/drugoverdose/maps/rxstate{year}.html'
    response = requests.get(cdc_url)
    soup = BeautifulSoup(response.content, 'lxml')

    tables = soup.find_all('table', attrs = {'class': 'table table-striped'})
    result_list = pd.read_html(str(tables[0]))
    world_soups = result_list[0]
    world_soups['Year']=year

    all_years=all_years.append(world_soups)

In [ ]:
all_years.head()

In [ ]:
len(all_years)

In [ ]:
## taking out NaN and making one column with state abbreviation
all_years['my_state']=np.nan
all_years['my_state']= all_years.my_state.fillna(all_years['State Abbreviation']).fillna(all_years.Abbreviation)


In [ ]:
all_years

In [ ]:
#Taking out extra columns
all_years=all_years[['State','Opioid Dispensing Rate per 100','Year','my_state']]

In [ ]:
all_years

In [ ]:
#saving to csv file
all_years.to_csv("DISP_RATE.csv")

## Bringing in Cause of Death by state data
### - Imported openpyxl above to load in an xlsx file
### - Data is for dates 1999-2019

In [56]:
all_years_cod_df = pd.read_excel('../data/icd10_grouped.xlsx',sheet_name = 0)

In [57]:
# reading in data, excluding notes at bottom (after rows 11831)
#all_years_cod_df = pd.read_excel('../data/Indiviual_years_Overdose.xlsx',nrows=11831, sheet_name = 0)

In [58]:
#making year not a float
#all_years_cod_df['Year'] = all_years_cod_df['Year'].astype(Int64Dtype())
all_years_cod_df['Year'] = pd.to_numeric(all_years_cod_df['Year'],errors='coerce').astype(pd.Int64Dtype())

In [59]:
#changing all suppressed values to 5
#the suppressed values are 0-9, a mathmetician suggested to use either 0 or half the value of 10
all_years_cod_df['Deaths'].mask(all_years_cod_df['Deaths'] == 'Suppressed', '5', inplace=True)
all_years_cod_df

,State,Year,Multiple Cause of death,Multiple Cause of death Code,Deaths,Population,Crude Rate
0,Alabama,1999,Heroin,T40.1,5,4430141,Suppressed
1,Alabama,1999,Other opioids,T40.2,12,4430141,Unreliable
2,Alabama,1999,Methadone,T40.3,16,4430141,Unreliable
3,Alabama,1999,Other synthetic narcotics,T40.4,11,4430141,Unreliable
4,Alabama,1999,Cocaine,T40.5,26,4430141,0.6
...,...,...,...,...,...,...,...
6421,Wyoming,2019,Other opioids,T40.2,31,578759,5.4
6422,Wyoming,2019,Methadone,T40.3,5,578759,Suppressed
6423,Wyoming,2019,Other synthetic narcotics,T40.4,18,578759,Unreliable
6424,Wyoming,2019,Cocaine,T40.5,5,578759,Suppressed


In [60]:
#dropping irrelevant columns
all_years_cod_df.drop('Multiple Cause of death Code',
  axis='columns', inplace=True)
all_years_cod_df.drop('Crude Rate',
  axis='columns', inplace=True)

In [61]:
#renaming row labels
all_years_cod_df=all_years_cod_df.replace(to_replace ="Other and unspecified narcotics",
                 value ="Other narcotics")

In [62]:
#renaming row labels
all_years_cod_df=all_years_cod_df.replace(to_replace ="Other synthetic narcotics",
                 value ="Other narcotics")

In [63]:
all_years_cod_df['Deaths'] = pd.to_numeric(all_years_cod_df['Deaths'],errors='coerce').astype(pd.Int64Dtype())

In [64]:
#group by other narcotics
#all_years_cod_df=all_years_cod_df.groupby(['State','Year','Multiple Cause of death','Population'],as_index=False).sum()
#all_years_cod_df=all_years_cod_df.groupby(['Deaths'])['Deaths'].sum()
#all_years_cod_df=all_years_cod_df.groupby(['State','Year','Multiple Cause of death']).agg({'Deaths':['sum']})
all_years_cod_df=all_years_cod_df.groupby(['State','Year','Multiple Cause of death','Population'],as_index=False)
all_years_cod_df=all_years_cod_df.aggregate(np.sum)
all_years_cod_df

,State,Year,Multiple Cause of death,Population,Deaths
0,Alabama,1999,Cocaine,4430141,26
1,Alabama,1999,Heroin,4430141,5
2,Alabama,1999,Methadone,4430141,16
3,Alabama,1999,Other narcotics,4430141,16
4,Alabama,1999,Other opioids,4430141,12
...,...,...,...,...,...
5350,Wyoming,2019,Cocaine,578759,5
5351,Wyoming,2019,Heroin,578759,12
5352,Wyoming,2019,Methadone,578759,5
5353,Wyoming,2019,Other narcotics,578759,23


In [ ]:
#adding crude rate column
all_years_cod_df['Crude Rate']=all_years_cod_df['Deaths']/(all_years_cod_df['Population']/100000)
all_years_cod_df

In [ ]:
all_years_cod_df=all_years_cod_df.astype({'Crude Rate':'float'})

In [ ]:
all_years_cod_df

In [ ]:
all_years_cod_df = all_years_cod_df.round({'Crude Rate': 2})

In [ ]:
all_years_cod_df

In [ ]:
all_years_cod_df.to_csv("CAUSE_OF_DEATH1.csv")

### Bringing in illicit vs rx COD

In [28]:
#this is age adjusted data
rx_cod = pd.read_excel('../data/illicit_rx_sep_files.xlsx', sheet_name = 0)
ill_cod = pd.read_excel('../data/illicit_rx_sep_files.xlsx', sheet_name = 1)
rx_synthetic=pd.read_excel('../data/illicit_rx_sep_files.xlsx', sheet_name = 2)

In [29]:
ill_cod['rx_ill']='IL'

In [30]:
ill_cod

,State,Year,Deaths,Population,Crude Rate,rx_ill
0,Alabama,1999,27,4430141,0.6,IL
1,Alabama,2000,19,4447100,Unreliable,IL
2,Alabama,2001,34,4467634,0.8,IL
3,Alabama,2002,33,4480089,0.7,IL
4,Alabama,2003,28,4503491,0.6,IL
...,...,...,...,...,...,...
1066,Wyoming,2015,Suppressed,586107,Suppressed,IL
1067,Wyoming,2016,14,585501,Unreliable,IL
1068,Wyoming,2017,15,579315,Unreliable,IL
1069,Wyoming,2018,14,577737,Unreliable,IL


In [31]:
rx_cod['rx_ill']='RX'
rx_cod

,State,Year,Deaths,Population,Crude Rate,rx_ill
0,Alabama,1999,30,4430141,0.7,RX
1,Alabama,2000,39,4447100,0.9,RX
2,Alabama,2001,61,4467634,1.4,RX
3,Alabama,2002,68,4480089,1.5,RX
4,Alabama,2003,49,4503491,1.1,RX
...,...,...,...,...,...,...
1066,Wyoming,2015,41,586107,7.0,RX
1067,Wyoming,2016,39,585501,6.7,RX
1068,Wyoming,2017,36,579315,6.2,RX
1069,Wyoming,2018,31,577737,5.4,RX


In [32]:
rx_synthetic['rx_ill']='SY'
rx_synthetic

,State,Year,Deaths,Population,Crude Rate,rx_ill
0,Alabama,1999,11,4430141,Unreliable,SY
1,Alabama,2000,13,4447100,Unreliable,SY
2,Alabama,2001,Suppressed,4467634,Suppressed,SY
3,Alabama,2002,Suppressed,4480089,Suppressed,SY
4,Alabama,2003,Suppressed,4503491,Suppressed,SY
...,...,...,...,...,...,...
1066,Wyoming,2015,Suppressed,586107,Suppressed,SY
1067,Wyoming,2016,Suppressed,585501,Suppressed,SY
1068,Wyoming,2017,18,579315,Unreliable,SY
1069,Wyoming,2018,10,577737,Unreliable,SY


### merging illicit and rx cod files

In [33]:
rx_ill_cod = pd.concat([rx_cod,ill_cod])
rx_ill_cod.sort_values(['State','Year'])

,State,Year,Deaths,Population,Crude Rate,rx_ill
0,Alabama,1999,30,4430141,0.7,RX
0,Alabama,1999,27,4430141,0.6,IL
1,Alabama,2000,39,4447100,0.9,RX
1,Alabama,2000,19,4447100,Unreliable,IL
2,Alabama,2001,61,4467634,1.4,RX
...,...,...,...,...,...,...
1068,Wyoming,2017,15,579315,Unreliable,IL
1069,Wyoming,2018,31,577737,5.4,RX
1069,Wyoming,2018,14,577737,Unreliable,IL
1070,Wyoming,2019,33,578759,5.7,RX


In [34]:
rx_ill_synth=pd.concat([rx_ill_cod,rx_synthetic])
rx_ill_synth.sort_values(['State','Year'])

,State,Year,Deaths,Population,Crude Rate,rx_ill
0,Alabama,1999,30,4430141,0.7,RX
0,Alabama,1999,27,4430141,0.6,IL
0,Alabama,1999,11,4430141,Unreliable,SY
1,Alabama,2000,39,4447100,0.9,RX
1,Alabama,2000,19,4447100,Unreliable,IL
...,...,...,...,...,...,...
1069,Wyoming,2018,14,577737,Unreliable,IL
1069,Wyoming,2018,10,577737,Unreliable,SY
1070,Wyoming,2019,33,578759,5.7,RX
1070,Wyoming,2019,25,578759,4.3,IL


In [42]:
#changing all suppressed values to 5
#the suppressed values are 0-9, a mathmetician suggested to use either 0 or half the value of 10
rx_ill_synth['Deaths'].mask(rx_ill_synth['Deaths'] == 'Suppressed', '5', inplace=True)
rx_ill_synth

,State,Year,Deaths,Population,Crude Rate,rx_ill
0,Alabama,1999,30,4430141,0.7,RX
1,Alabama,2000,39,4447100,0.9,RX
2,Alabama,2001,61,4467634,1.4,RX
3,Alabama,2002,68,4480089,1.5,RX
4,Alabama,2003,49,4503491,1.1,RX
...,...,...,...,...,...,...
1066,Wyoming,2015,5,586107,Suppressed,SY
1067,Wyoming,2016,5,585501,Suppressed,SY
1068,Wyoming,2017,18,579315,Unreliable,SY
1069,Wyoming,2018,10,577737,Unreliable,SY


In [43]:
rx_ill_synth=rx_ill_synth.astype({'Deaths':'int64'})

In [44]:
rx_ill_synth['Crude Rate']=rx_ill_synth['Deaths']/(rx_ill_synth['Population']/100000)
rx_ill_synth

,State,Year,Deaths,Population,Crude Rate,rx_ill
0,Alabama,1999,30,4430141,0.677179,RX
1,Alabama,2000,39,4447100,0.876976,RX
2,Alabama,2001,61,4467634,1.365376,RX
3,Alabama,2002,68,4480089,1.517827,RX
4,Alabama,2003,49,4503491,1.088045,RX
...,...,...,...,...,...,...
1066,Wyoming,2015,5,586107,0.853087,SY
1067,Wyoming,2016,5,585501,0.853970,SY
1068,Wyoming,2017,18,579315,3.107118,SY
1069,Wyoming,2018,10,577737,1.730891,SY


In [45]:
#rx_ill_cod=rx_ill_cod['Crude Rate'].round(decimals=2)
#rx_ill_cod=rx_ill_cod.round(2)
#rx_ill_cod['Crude Rate']=rx_ill_cod['Crude Rate'].round()
#rx_ill_cod['Crude Rate'] = rx_ill_cod['Crude Rate'].apply(lambda x: round(x, 2))
rx_ill_synth = rx_ill_synth.round({'Crude Rate': 2})

In [46]:
rx_ill_synth

,State,Year,Deaths,Population,Crude Rate,rx_ill
0,Alabama,1999,30,4430141,0.68,RX
1,Alabama,2000,39,4447100,0.88,RX
2,Alabama,2001,61,4467634,1.37,RX
3,Alabama,2002,68,4480089,1.52,RX
4,Alabama,2003,49,4503491,1.09,RX
...,...,...,...,...,...,...
1066,Wyoming,2015,5,586107,0.85,SY
1067,Wyoming,2016,5,585501,0.85,SY
1068,Wyoming,2017,18,579315,3.11,SY
1069,Wyoming,2018,10,577737,1.73,SY


In [47]:
rx_ill_synth.to_csv("RX_ILL_SYNTH.csv")

### Creating a dataframe when PDMP started

In [ ]:
pdmp=pd.read_csv('../data/PDMP_info.csv',usecols=['Jurisdiction','1.4. When did the PDMP start receiving prescription data electronically?'])

In [ ]:
pdmp

In [ ]:
#renaming columns
pdmp=pdmp.rename(columns={'Jurisdiction':'State','1.4. When did the PDMP start receiving prescription data electronically?':'Year'},inplace=False)

In [ ]:
#changing year to datetime
pdmp['Year']=pd.to_datetime(pdmp['Year'])

In [ ]:
#extracting year from date
pdmp['Year'] = pdmp['Year'].dt.year

In [ ]:
pdmp.to_csv("PDMP.csv")

### Creating a dataframe for laws